In [1]:
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U transformers
!pip install -U datasets


   --------------------------------------- 121.5/121.5 MB 10.4 MB/s eta 0:00:00


  Using cached tokenizers-0.20.0-cp38-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 9.9/9.9 MB 11.0 MB/s eta 0:00:00
Using cached tokenizers-0.20.0-cp38-none-win_amd64.whl (2.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 0.8.2 requires accelerate<0.17,>=0.9, but you have accelerate 0.34.2 which is incompatible.
autogluon-multimodal 0.8.2 requires Pillow<9.6,>=9.3, but you have pillow 10.2.0 which is incompatible.
autogluon-multimodal 0.8.2 requires scikit-image<0.20.0,>=0.19.1, but you have scikit-image 0.21.0 which is incompatible.
autogluon-multimodal 0.8.2 requires torch<1.14,>=1.9, but you have torch 2.2.2 which is incompatible.
autogluon-multimodal 0.8.2 requires torchvision<0.15.0, but you have torchvision 0.17.2 which is incompatible.
autogluon-multimodal 0.8.2 requires transformers[sentencepiece]<4.27.0,>=4.23.0, but you have transformers 4.45.1 which is incompatible.


In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

: 

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16)

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'google/gemma-2-2b'

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

C:\Users\nice2\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nice2\.cache\huggingface\hub\models--google--gemma-2-2b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplicati

RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend